In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilities.OneHotEncoder import OneHotEncoder
from utilities.DataPreProcessor import DataPreProcessor

In [2]:
df_pdb_data_no_seq = pd.read_csv('resources/pdb_data_no_dups.csv')
df_pdb_seq = pd.read_csv('resources/pdb_data_seq.csv')
print("..Read data files..")

..Read data files..


In [3]:
# filtering out only protein structures
df_protein_seq = df_pdb_seq[(df_pdb_seq['macromoleculeType'] == 'Protein')]

In [4]:
# joining two data sets for protein Seq - classification

df_protein_final = df_protein_seq.merge(df_pdb_data_no_seq)

df_protein_seq_final = df_protein_final[['sequence', 'classification']]

df_protein_seq_final.head()

,sequence,classification
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
1,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,HYDROLASE(O-GLYCOSYL)
2,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
3,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,HYDROLASE(O-GLYCOSYL)
4,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT


In [5]:
# drop na values in the data frame

df_protein_seq_final = df_protein_seq_final.dropna()
print("filtered data with NA values")

filtered data with NA values


In [6]:
##One_hot encoding

one_hot_encoder = OneHotEncoder()

one_hot_encoder.find_unique_amino_acids(df_protein_seq_final['sequence'])
print("Unique Amino Acid Set...")

Unique Amino Acid Set...


In [7]:
## creates an encoded dictionaryfor each Amino Acid
one_hot_encoder.create_one_hot_dict()

data_pre_processor = DataPreProcessor(one_hot_encoder,
                                      df_protein_seq_final['classification']
                                      .value_counts().to_dict())

In [8]:
## delete least preferred sequences
data_pre_processor.del_least_preferred()

df_protein_seq_final_for_modeling = \
    df_protein_seq_final[df_protein_seq_final['classification']
        .isin(data_pre_processor.count_dict_keys_as_list())]

df_protein_seq_final_for_modeling = \
    df_protein_seq_final_for_modeling.reset_index(drop=True)
print("final Shape of the DataSet {}".format(df_protein_seq_final_for_modeling.shape))

final Shape of the DataSet (291355, 2)


In [9]:
X_sequences = df_protein_seq_final_for_modeling['sequence']
y_label = df_protein_seq_final_for_modeling['classification']

In [10]:
## test process sequences

data_pre_processor.process_all_seqs(X_sequences[:10000], y_label[:10000])
# print("Getting Processed Seqs")
# print(data_pre_processor.get_x())
# print("getting labels")
# print(data_pre_processor.get_y())
print("getting protein categories")
print(data_pre_processor.get_protein_categoeies())

getting protein categories
['ANTIMICROBIAL PROTEIN', 'APOPTOSIS', 'BIOSYNTHETIC PROTEIN', 'BLOOD CLOTTING', 'CELL ADHESION', 'CELL CYCLE', 'CHAPERONE', 'CONTRACTILE PROTEIN', 'CYTOKINE', 'DE NOVO PROTEIN', 'DNA BINDING PROTEIN', 'ELECTRON TRANSPORT', 'FLUORESCENT PROTEIN', 'GENE REGULATION', 'HORMONE', 'HORMONE/GROWTH FACTOR', 'HYDROLASE', 'HYDROLASE/HYDROLASE INHIBITOR', 'Hydrolase/Hydrolase Inhibitor', 'IMMUNE SYSTEM', 'ISOMERASE', 'LIGASE', 'LIPID BINDING PROTEIN', 'LYASE', 'LYASE/LYASE INHIBITOR', 'MEMBRANE PROTEIN', 'METAL BINDING PROTEIN', 'METAL TRANSPORT', 'MOTOR PROTEIN', 'OXIDOREDUCTASE', 'OXIDOREDUCTASE/ELECTRON TRANSPORT', 'OXIDOREDUCTASE/OXIDOREDUCTASE INHIBITOR', 'OXYGEN STORAGE/TRANSPORT', 'OXYGEN TRANSPORT', 'PEPTIDE BINDING PROTEIN', 'PHOTOSYNTHESIS', 'PLANT PROTEIN', 'PROTEIN BINDING', 'PROTEIN FIBRIL', 'PROTEIN TRANSPORT', 'REPLICATION', 'RNA BINDING PROTEIN', 'SIGNALING PROTEIN', 'STRUCTURAL GENOMICS, UNKNOWN FUNCTION', 'STRUCTURAL PROTEIN', 'SUGAR BINDING PROTEIN',

### LSTM Modeling

In [11]:
# from model.LSTMModel import LSTMModel

# lstm_model = LSTMModel(None,61)

In [12]:
# print(lstm_model.define_model(100,3))

In [13]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_label, test_size=0.20, random_state=42)

In [14]:
# for X,y in data_pre_processor.batch_generator(512):
#     #print(y.shape)
#     lstm_model.get_model().fit(X,y,epochs=10,verbose=2)

In [15]:
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout

Using TensorFlow backend.


In [16]:
aa_size = 25
num_features = 25

model = Sequential()
#model.add(Embedding(aa_size,100,input_length=None))
model.add(LSTM(25,input_shape=(None, num_features)))
model.add(Dropout(0.2))
model.add(Dense(len(data_pre_processor.get_protein_categoeies()),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 57)                1482      
Total params: 6,582
Trainable params: 6,582
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
for X,y in data_pre_processor.batch_generator(64):
    #print(X.shape)
    model.fit(X,y,epochs=2,verbose=2,validation_split=0.25)

Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 4.0421 - acc: 0.1042 - val_loss: 4.0464 - val_acc: 0.3125
Epoch 2/2
 - 0s - loss: 4.0362 - acc: 0.2708 - val_loss: 4.0424 - val_acc: 0.2500
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 4.0387 - acc: 0.0625 - val_loss: 4.0207 - val_acc: 0.5000
Epoch 2/2
 - 1s - loss: 4.0345 - acc: 0.1250 - val_loss: 4.0142 - val_acc: 0.5000
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 4.0283 - acc: 0.0417 - val_loss: 4.0339 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 4.0221 - acc: 0.0625 - val_loss: 4.0292 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 4.0176 - acc: 0.0625 - val_loss: 4.0325 - val_acc: 0.1250
Epoch 2/2
 - 1s - loss: 4.0100 - acc: 0.1250 - val_loss: 4.0301 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 0s - loss: 4.0088 - acc: 0.1250 - val_loss: 3.9980 - val_acc: 0.6250
Epoch 2/2
 - 0s - loss: 4.0015

Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.8818 - acc: 0.0833 - val_loss: 2.2769 - val_acc: 0.8125
Epoch 2/2
 - 1s - loss: 2.8248 - acc: 0.1875 - val_loss: 2.2536 - val_acc: 0.8125
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.0992 - acc: 0.1667 - val_loss: 3.0571 - val_acc: 0.1250
Epoch 2/2
 - 1s - loss: 3.1256 - acc: 0.2292 - val_loss: 3.0607 - val_acc: 0.1250
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.7100 - acc: 0.1250 - val_loss: 3.3313 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 2.7477 - acc: 0.1250 - val_loss: 3.3456 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.5729 - acc: 0.0417 - val_loss: 3.1819 - val_acc: 0.2500
Epoch 2/2
 - 1s - loss: 3.5923 - acc: 0.0208 - val_loss: 3.1803 - val_acc: 0.2500
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.2799 - acc: 0.1042 - val_loss: 3.5785 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 3.2576

Epoch 2/2
 - 1s - loss: 2.6759 - acc: 0.1250 - val_loss: 2.4207 - val_acc: 0.1250
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.5604 - acc: 0.4375 - val_loss: 2.7507 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 2.5198 - acc: 0.4167 - val_loss: 2.7437 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.3082 - acc: 0.2292 - val_loss: 2.4251 - val_acc: 0.5000
Epoch 2/2
 - 1s - loss: 3.3031 - acc: 0.2292 - val_loss: 2.4207 - val_acc: 0.5000
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.6571 - acc: 0.2917 - val_loss: 1.7559 - val_acc: 0.5625
Epoch 2/2
 - 1s - loss: 2.7207 - acc: 0.2708 - val_loss: 1.7356 - val_acc: 0.5625
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.6296 - acc: 0.3750 - val_loss: 2.2466 - val_acc: 0.3750
Epoch 2/2
 - 1s - loss: 2.5592 - acc: 0.3333 - val_loss: 2.2330 - val_acc: 0.3750
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.8190 - a

Epoch 2/2
 - 1s - loss: 3.3841 - acc: 0.0417 - val_loss: 2.7483 - val_acc: 0.2500
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 0s - loss: 3.0462 - acc: 0.2500 - val_loss: 3.3153 - val_acc: 0.0000e+00
Epoch 2/2
 - 0s - loss: 3.0884 - acc: 0.2292 - val_loss: 3.3146 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 0s - loss: 3.3967 - acc: 0.1250 - val_loss: 3.7447 - val_acc: 0.1250
Epoch 2/2
 - 0s - loss: 3.3485 - acc: 0.1042 - val_loss: 3.7433 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.4848 - acc: 0.0417 - val_loss: 3.6056 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 3.4559 - acc: 0.1042 - val_loss: 3.6049 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 2.9231 - acc: 0.2083 - val_loss: 3.6884 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss: 2.9025 - acc: 0.1458 - val_loss: 3.6870 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1

 - 0s - loss: 2.3750 - acc: 0.3750 - val_loss: 2.7398 - val_acc: 0.0000e+00
Epoch 2/2
 - 0s - loss: 2.3425 - acc: 0.3125 - val_loss: 2.6747 - val_acc: 0.0000e+00
Train on 12 samples, validate on 4 samples
Epoch 1/2
 - 0s - loss: 2.1519 - acc: 0.3333 - val_loss: 1.8227 - val_acc: 0.0000e+00
Epoch 2/2
 - 0s - loss: 2.3158 - acc: 0.0000e+00 - val_loss: 1.8117 - val_acc: 0.0000e+00


AttributeError: 'list' object has no attribute 'reshape'

In [43]:
count = 0
for i in range(1000,2000,1):
    yhat = model.predict(np.array(data_pre_processor.get_x()[i])
                         .reshape(1,np.array(data_pre_processor.get_x()[i]).shape[0]
                                  ,np.array(data_pre_processor.get_x()[i]).shape[1]))
    
    act_y = np.array(data_pre_processor.get_y())[i]
    
    if np.argmax(yhat) == np.argmax(act_y):
        count += 1

print("Accuracy is {}".format(count))

Accuracy is 162


23

16